In [13]:
from trackml.dataset import load_event
from trackml.dataset import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from tqdm import tqdm
from multiprocessing import Pool
from sklearn.neighbors import KDTree

In [30]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 1:
        break
    cnt += 1

In [42]:
10 * 200 / 60 / 8

4.166666666666667

In [14]:
	def extend(submission,hits):

		df = submission.merge(hits,  on=['hit_id'], how='left')
		df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
		df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
		df = df.assign(arctan2 = np.arctan2(df.z, df.r))

		for angle in range(-180,180,1):

		    #print ('\r %f'%angle, end='',flush=True)
		    #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
		    df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

		    min_num_neighbours = len(df1)
		    if min_num_neighbours<4: continue

		    hit_ids = df1.hit_id.values
		    x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
		    r  = (x**2 + y**2)**0.5
		    r  = r/1000
		    a  = np.arctan2(y,x)
		    tree = KDTree(np.column_stack([a,r]), metric='euclidean')

		    track_ids = list(df1.track_id.unique())
		    num_track_ids = len(track_ids)
		    min_length=3

		    for i in range(num_track_ids):
		        p = track_ids[i]
		        if p==0: continue

		        idx = np.where(df1.track_id==p)[0]
		        if len(idx)<min_length: continue

		        if angle>0:
		            idx = idx[np.argsort( z[idx])]
		        else:
		            idx = idx[np.argsort(-z[idx])]


		        ## start and end points  ##
		        idx0,idx1 = idx[0],idx[-1]
		        a0 = a[idx0]
		        a1 = a[idx1]
		        r0 = r[idx0]
		        r1 = r[idx1]

		        da0 = a[idx[1]] - a[idx[0]]  #direction
		        dr0 = r[idx[1]] - r[idx[0]]
		        direction0 = np.arctan2(dr0,da0) 

		        da1 = a[idx[-1]] - a[idx[-2]]
		        dr1 = r[idx[-1]] - r[idx[-2]]
		        direction1 = np.arctan2(dr1,da1) 

	 
		        ## extend start point
		        ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)
		        direction = np.arctan2(r0-r[ns],a0-a[ns])
		        ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]
	   
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

		        ## extend end point
		        ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)

		        direction = np.arctan2(r[ns]-r1,a[ns]-a1)
		        ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 
		        
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
		#print ('\r')
		df = df[['event_id', 'hit_id', 'track_id']]
		return df


In [15]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [16]:
def merge(cl1, cl2, min_cnt): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) &  (d['N2'].values>min_cnt))
    #cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) )

    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 
    return s1

In [14]:
z_result = np.load('predicts/200_zshift.npy')
z_result = z_result[:, 0, :]

result_53 = np.load('predicts/53_all.npy')
result_53 = np.array(result_53)

luis = np.load('predicts/luis_all.npy')

In [18]:
result = np.load('predicts/luis_all_1000_event.npy')

In [30]:
result_53 = np.load('predicts/53/0.npy')
result_53 = np.array(result_53)

In [31]:
result_53.shape

(2000, 100287)

In [20]:
result.shape

(3000, 120939)

In [17]:
luis.shape

(338, 100287)

In [68]:
result_53 = np.load('predicts/53_all.npy')
result_luis = np.load('predicts/luis_all.npy')

In [62]:
result = np.load('predicts/luis_all_13k.npy')

In [7]:
os.listdir('predicts/test_1')

['event_8.npy',
 'event_28.npy',
 'event_93.npy',
 'event_54.npy',
 'event_118.npy',
 'event_96.npy',
 'event_7.npy',
 'event_29.npy',
 'event_102.npy',
 'event_62.npy',
 'event_66.npy',
 'event_18.npy',
 'event_99.npy',
 'event_16.npy',
 'event_76.npy',
 'event_58.npy',
 'event_4.npy',
 'event_51.npy',
 'event_98.npy',
 'event_60.npy',
 'event_49.npy',
 'event_2.npy',
 'event_95.npy',
 'event_43.npy',
 'event_81.npy',
 'event_94.npy',
 'event_86.npy',
 'event_57.npy',
 'event_15.npy',
 'event_11.npy',
 'event_71.npy',
 'event_89.npy',
 'event_56.npy',
 'event_25.npy',
 'event_44.npy',
 'event_87.npy',
 'event_5.npy',
 'event_101.npy',
 'event_46.npy',
 'event_91.npy',
 'event_74.npy',
 'event_68.npy',
 'event_107.npy',
 'event_3.npy',
 'event_123.npy',
 'event_27.npy',
 'event_42.npy',
 'event_119.npy',
 'event_47.npy',
 'event_80.npy',
 'event_55.npy',
 'event_85.npy',
 'event_82.npy',
 'event_37.npy',
 'event_64.npy',
 'event_124.npy',
 'event_14.npy',
 'event_33.npy',
 'event_23.np

In [8]:
result = np.load('predicts/test_1/event_0.npy')

In [69]:
result = result_luis

In [71]:
result = np.array(result_luis)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

NameError: name 'hits' is not defined

In [ ]:
0.5977835924474594
0.5683607021832576

In [21]:
from threading import Lock
global lock
lock = Lock()

In [24]:
def deeper(i):
    #print(i)
    np.random.seed(i)
    #result = p
    lock.acquire()
    np.random.shuffle(result)
    lock.release()
    labels = range(result.shape[1])

    for k in [0]:
        for i in range(len(result[:])):
            labels = merge(labels, result[i], k)

        #submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        #print(score_event(truth, submission))
    return labels

In [55]:
from itertools import permutations
pairs = []
for i in tqdm(np.unique(result[0])):
    idxs = np.where(result[0] == i)[0]
    for a, b in permutations(idxs, 2):
        pairs += [[a, b]]
        M[a, b] = 1

  0%|          | 0/94941 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
100%|██████████| 94941/94941 [02:06<00:00, 750.60it/s]


In [59]:
M.nonzero()

(array([     0,      1,      4, ..., 120926, 120935, 120938], dtype=int32),
 array([     0,   1419,     37, ..., 120918, 120442, 120449], dtype=int32))

In [65]:
M[M.nonzero()]

matrix([[1, 1, 1, ..., 1, 1, 1]], dtype=int8)

In [54]:
np.array(pairs).shape

(97168, 2)

In [50]:
idxs

[array([0]),
 array([   1, 1419]),
 array([2]),
 array([3]),
 array([ 4, 37]),
 array([ 5, 42]),
 array([    6,  1416,  1449,  3182,  3224,  5362,  5387,  7867,  7964,
        10757, 10838, 10900, 13801]),
 array([ 7, 47]),
 array([   8, 1471, 3222]),
 array([9]),
 array([  10, 1418]),
 array([11]),
 array([  12, 1425]),
 array([  13, 1422]),
 array([  14, 1426]),
 array([15]),
 array([16]),
 array([17]),
 array([  18, 1431]),
 array([  19, 1417]),
 array([   20,    44,  1451,  1510,  3239,  3278,  5408,  5486,  7946,
         8030, 10774, 10909, 13818]),
 array([  21, 1443]),
 array([   22,  1445,  1447,  3178,  3195,  5337,  5347,  7874,  7879,
        10758, 10800, 10827, 10860, 13722, 13752, 13795, 13814, 13869,
        13896, 16790, 16792, 16820, 16833]),
 array([23]),
 array([   24,  1479,  3263,  5420,  7934, 10782, 10855, 13825, 13904,
        16811]),
 array([25]),
 array([26]),
 array([27]),
 array([28, 53]),
 array([29]),
 array([30]),
 array([31]),
 array([   32,  1456,  32

In [33]:
result.shape

(3000, 120939)

In [67]:
from scipy.sparse import csr_matrix
M = csr_matrix((120939, 120939), dtype=np.int8)

In [36]:
M[0, 0] += 1

/usr/local/lib/python3.5/dist-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [39]:
M

<120939x120939 sparse matrix of type '<class 'numpy.int8'>'
	with 1 stored elements in Compressed Sparse Row format>

In [36]:
result = np.vstack([result_53, result_luis])


In [25]:
pool = Pool(processes=8)
second = []
for i in tqdm(pool.imap(deeper, range(10))):
    second += [i]
pool.close()
second = np.array(second)


0it [00:00, ?it/s]
1it [02:41, 161.66s/it]
2it [02:44, 82.07s/it] 
9it [03:38, 24.28s/it]
10it [03:38, 21.86s/it]

In [29]:
np.save('predicts/1000_event.py', second)

In [56]:
result = np.array(second)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.5839686423543643


In [57]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6006789191280288


In [58]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6055490242024659


In [50]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6332634287844977


In [51]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6384495204723458


In [52]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6389822860343759


In [62]:
second = []
for k in range(10):
    result = np.vstack([result_53, result_luis])
    np.random.shuffle(result)

    #result = res0
    labels = range(result.shape[1])

    for k in [0]:
        for i in range(len(result[:])):
            labels = merge(labels, result[i], k)

        submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        print(score_event(truth, submission))
        
    second += [labels]
    

0.5627779192433786
0.5617721875345107
0.5634307926550272
0.5615686496268391
0.5629473823233996
0.5624647783273607
0.5637125711193076
0.5617281078810507
0.560673030178837
0.5620484298484023


In [64]:
result = np.array(second)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.5793362234676419


In [65]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6020060898369067


In [66]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6073700798358106


In [67]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6084732685060847


In [68]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6089716249422316


In [ ]:
0.5464415834327985
0.5452367041039137
0.5117532610647899

In [ ]:
0.6084732685060847

In [15]:
import numpy as np

In [19]:
np.load('predicts/event_0.npy').shape

(10, 119016)

In [20]:
np.load('predicts/event_0.npy')

array([[  2858059,  12820535,    119017, ..., 272775373, 275123875,
        274169210],
       [   929991,  10930814,    119017, ...,  20191763, 268634766,
        274638418],
       [   381723, 250572701,    119017, ..., 275908894, 179724814,
        273729412],
       ...,
       [   568198, 240070401,    119017, ..., 275263310, 270618380,
        273643712],
       [  3838831,  31216402,    119017, ..., 267654876, 270035751,
        271840477],
       [   891931,  14569677,    119017, ..., 274982333, 218622234,
        275567520]])

In [16]:
tot = np.load('predicts/total.npy')


In [17]:
tot.shape

(201, 100287)

In [18]:
second = []
for k in range(10):
    result = tot
    np.random.shuffle(result)

    #result = res0
    labels = range(result.shape[1])

    for k in [0]:
        for i in range(len(result[:])):
            labels = merge(labels, result[i], k)

        submission = create_one_event_submission(0, hits['hit_id'].values, labels)
        submission = extend(submission,hits)
        labels = submission['track_id'].values
        print(score_event(truth, submission))
        
    second += [labels]

0.6181009325698372
0.6185834646082384
0.609313002578232
0.6143053976691017
0.6202792246183331
0.6138796066679132
0.6145178940037926
0.6156359163813985
0.6176519920828759
0.6172558849625269


In [19]:
result = np.array(second)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.6289796583597507


In [11]:
result = np.array(tot)
labels = range(result.shape[1])

for k in [0]:
    for i in range(len(result[:])):
        labels = merge(labels, result[i], k)

    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.6015647245638243


In [20]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6283711921504336


In [21]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6293434027846148


In [22]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6308982276201209


In [23]:
submission = extend(submission,hits)

print(score_event(truth, submission))

0.6302840939445985


In [ ]:
0.6308982276201209

In [ ]:
submission = create_one_event_submission(0, hits['hit_id'].values, labels)


In [24]:
200*300

60000

In [28]:
10*200 / 60 /32

1.0416666666666667

In [32]:
0.000005 * 300

0.0015

In [362]:
#result = result_53
labels = range(result.shape[1])

for k in [18,16,14, 12, 10, 8, 6, 4, 0]:
#for k in [15,10,5, 0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    #submission = extend(submission,hits)
    #submission = extend(submission,hits)

    print(score_event(truth, submission))
    
    labels = submission['track_id'].values
   

0.02946572701478705
0.02946572701478705
0.05727761228580784
0.05727761228580784
0.11534065410312926
0.11534065410312926
0.23344574246875743
0.23344574246875743
0.36729231106049864
0.36729231106049864
0.45506391156244447
0.45506391156244447
0.4944932151899727
0.4944932151899727
0.5049299479307393
0.5049299479307393
0.5073244085906707
0.5073244085906707


NameError: name 'np' is not defined

In [329]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5848926941183028


In [330]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5850779423213555


In [325]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5850817906028537


In [326]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5853420574657813


In [327]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5855217023763549


In [352]:
labels1 = merge(labels_luis, labels, 0)
submission = create_one_event_submission(0, hits['hit_id'].values, labels1)
print(score_event(truth, submission))

0.4913849820080143


In [ ]:
0.5076282691673919
0.5067205748968073
0.4731231005334312

In [ ]:
0.4912658962859528

In [ ]:
result = result_53
labels = range(z_result.shape[1])

for k in [15,10,5, 0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    submission = extend(submission,hits)
    print(score_event(truth, submission))
    
    labels = submission['track_id'].values
   

In [303]:
submission = create_one_event_submission(0, hits['hit_id'].values, labels)

In [307]:
score_event(truth, ext_sub)

0.5102744886009511

In [ ]:
0.5855217023763549
0.5853420574657813
0.5850817906028537
0.5849290849426552
0.5827447401532195
0.5810983311646127
0.5807878755572605
0.5792614621063498
0.5102744886009511

In [39]:
np.sqrt(100000)

316.22776601683796